In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.integrate import cumulative_trapezoid
from pysrim import SR, Ion, Layer, Target

In [ ]:
ion = Ion('H', energy=45e6)

layer = Layer(
    elements={
        'Al': {
            'stoich': 1.0,
            'E_d':     25.0, # Displacement energy [eV], common default
            'lattice':  0.0, # Lattice binding energy [eV]
            'surface': 3.36  # Surface binding energy [eV]
        }
    },
    density=2.70,          # Aluminium density in g/cm^3
    width=0,             # 1 mm in Angstroms
    name="Aluminium"
)

target = Target([layer])

srim = SR(layer, ion, output_type=5) # MeV·cm^2/mg
#    output_type:
#    (1) eV/Angstrom
#    (2) keV/micron
#    (3) MeV/mm
#    (4) keV / (ug/cm2)
#    (5) MeV / (mg/cm2)
#    (6) keV / (mg/cm2)
#    (7) eV / (1E15 atoms/cm2)
#    (8) L.S.S reduced units

srim_executable_directory = '/Users/mdovale/Applications/srim' 

results = srim.run(srim_executable_directory)

x = 1e-3*results.data[0]/results.ion['A1']

plt.plot(x, results.data[1], label='Electronic stopping power')
# plt.plot(x, results.data[2], label='Nuclear stopping power')
# plt.plot(x, results.data[3], label='Projected range (um)')
# plt.plot(x, results.data[4], label='Longitudinal straggling (um)')
# plt.plot(x, results.data[5], label='Lateral straggling (um)')=
plt.legend()
plt.xlabel('MeV/u')
plt.ylabel(results.units)
plt.title(results.ion['name']+' in '+str(list(results.target.values())[-1]))
plt.xscale('log')
plt.grid()
plt.show()

In [ ]:
# --------------------------------------
# Load SRIM stopping power table
# --------------------------------------
# E: Proton energy values [keV] from SRIM; convert to MeV
E = np.array(results.data[0]) * 1e-3  # MeV

# S: Electronic stopping power [MeV·cm^2/mg] from SRIM
S = np.array(results.data[1])  # MeV·cm^2/mg

# --------------------------------------
# Convert stopping power to MeV/mm
# --------------------------------------
# Aluminum density: 2.7 g/cm^3 = 2700 mg/cm^3
density = 2700  # mg/cm^3

# Convert stopping power from MeV·cm^2/mg to MeV/cm, then to MeV/mm
S_mm = S * density / 10  # MeV/mm

# --------------------------------------
# Reverse arrays for integration (start from high energy to low)
# --------------------------------------
E = E[::-1]
S_mm = S_mm[::-1]

# --------------------------------------
# Integrate 1/S(E) to compute CSDA range: depth [mm] as a function of energy
# --------------------------------------
# Integration gives cumulative depth (path length) for each energy
depth_mm = -cumulative_trapezoid(1 / S_mm, E, initial=0)  # in mm

# --------------------------------------
# Interpolate energy as a function of depth
# --------------------------------------
# This gives the residual energy of a proton after passing through a given depth of material
E_vs_depth = interp1d(depth_mm, E, kind='linear', bounds_error=False)

# --------------------------------------
# Example: compute residual energy after d mm of Aluminum
# --------------------------------------
d = 1.0  # mm
residual_energy = E_vs_depth(d)
print(f"Energy after {d:.2f} mm Al: {residual_energy:.2f} MeV")

In [ ]:
depths_mm = np.linspace(0,10,100)
residual_energies = E_vs_depth(depths_mm)
plt.plot(depth_mm, E)
plt.plot(depths_mm, residual_energies, ls='--')
plt.xlabel("Depth in Al (cm)")
plt.ylabel("Proton Energy (MeV)")
plt.title("Proton Energy vs. Depth in Aluminum")
plt.grid(True)
plt.show()